# Flat Image Net - Basic Graph


 - fix loss
 - fix optimizer
 - use train and valid
 - extend summaries
 - add relu activation
 - inspect distribution of values
 - look at predictions on valid

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import shutil
from utils.data import init_dir

## Load data

In [3]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

Extracting /data/fashion/train-images-idx3-ubyte.gz
Extracting /data/fashion/train-labels-idx1-ubyte.gz
Extracting /data/fashion/t10k-images-idx3-ubyte.gz
Extracting /data/fashion/t10k-labels-idx1-ubyte.gz


## Basic summary functions

In [4]:
def variable_summaries(name, var):
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))

        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


def img_summaries(name, var):
    with tf.name_scope(name):
        tf.summary.image(name, var)

## Custom layer functions

In [5]:
def simple_layer(name, input_data, shape, activation='linear'):
    w_name = 'w_' + name
    b_name = 'b_' + name
    if activation == 'relu':
        w = tf.get_variable(w_name, shape=shape, initializer=tf.contrib.layers.variance_scaling_initializer())
    else:
        #w = tf.get_variable('w', initializer=tf.truncated_normal(shape, stddev=0.1))
        w = tf.get_variable(w_name, shape=shape, initializer=tf.contrib.layers.xavier_initializer())
    bias = tf.get_variable(b_name, initializer=tf.constant_initializer(0.1), shape=shape[1])
    
    variable_summaries(w_name+'summary', w)
    variable_summaries(b_name+'summary', bias)
    
    output_data = tf.matmul(input_data, w) + bias
    if activation == 'relu':
        output_data = tf.nn.relu(output_data)
    elif activation == 'sigmoid':
        output_data = tf.nn.sigmoid(output_data)
    return output_data

## Build Net Graph

In [6]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('flat_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
    
    with tf.variable_scope('simple_layer_1'):
        raw_prediction = simple_layer(name='layer1', input_data=images, shape=[784, 64], activation='relu')
        
    with tf.variable_scope('simple_layer_2'):
        raw_prediction = simple_layer(name='layer2', input_data=raw_prediction, shape=[64, 10])
            
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(raw_prediction)
    
    with tf.name_scope('loss'):
        cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=prediction)
        loss = tf.reduce_mean(cross_entropy_vector)
        variable_summaries('loss_summary', loss)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)       
        
    with tf.name_scope('training'):
        train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
            
    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

## Init Model Logging

In [7]:
from utils.data import init_model_logging
base_dir = '/tensorboard_summaries/flat_image_net/'

logging_meta = init_model_logging(base_dir, 'experiment1', graph=graph, remove_existing=True)
logging_meta

{'model_path': '/tensorboard_summaries/flat_image_net/experiment1/valid/model.ckpt',
 'saver': <tensorflow.python.training.saver.Saver at 0x7f8d0f786828>,
 'train_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7f8d0f4d0fd0>,
 'train_writer_dir': '/tensorboard_summaries/flat_image_net/experiment1/train',
 'valid_writer': <tensorflow.python.summary.writer.writer.FileWriter at 0x7f8d0f786a58>,
 'valid_writer_dir': '/tensorboard_summaries/flat_image_net/experiment1/valid'}

## Run Net

In [8]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']
        
with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    for iteration in range(10000):
        _images, _labels = data.train.next_batch(100)
        
        _ = session.run([train_step], feed_dict={images: _images, labels: _labels})
        
        if iteration % 10 == 0:
            _summary, = session.run([merge_summaries],feed_dict={images: _images, labels: _labels})
            logging_meta['train_writer'].add_summary(_summary, iteration)    
        if iteration % 100 == 0:
            _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss], 
                                          feed_dict={images: data.validation.images, 
                                                     labels: data.validation.labels})
            logging_meta['valid_writer'].add_summary(_summary, iteration)
            logging_meta['saver'].save(session, model_path, iteration)
            print("Iteration {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))
            
    _prediction, = session.run([prediction], feed_dict={images: data.validation.images})

Iteration 0: loss 2.2908389568328857, accuracy 0.15320000052452087
Iteration 100: loss 1.7984511852264404, accuracy 0.6901999711990356
Iteration 200: loss 1.7316980361938477, accuracy 0.7459999918937683
Iteration 300: loss 1.7100551128387451, accuracy 0.7626000046730042
Iteration 400: loss 1.7042183876037598, accuracy 0.7681999802589417
Iteration 500: loss 1.6931500434875488, accuracy 0.7771999835968018
Iteration 600: loss 1.6912761926651, accuracy 0.7770000100135803
Iteration 700: loss 1.6881932020187378, accuracy 0.7797999978065491
Iteration 800: loss 1.6856456995010376, accuracy 0.7814000248908997
Iteration 900: loss 1.6903926134109497, accuracy 0.777999997138977
Iteration 1000: loss 1.6791906356811523, accuracy 0.7853999733924866
Iteration 1100: loss 1.6751341819763184, accuracy 0.7904000282287598
Iteration 1200: loss 1.6793400049209595, accuracy 0.7850000262260437
Iteration 1300: loss 1.6708614826202393, accuracy 0.7943999767303467
Iteration 1400: loss 1.6683952808380127, accuracy

In [9]:
_prediction

array([[  2.11483678e-30,   4.91579752e-28,   8.11237275e-32, ...,
          4.14084897e-14,   1.78988800e-23,   1.00000000e+00],
       [  1.00000000e+00,   1.96968322e-28,   1.37922157e-20, ...,
          7.22811649e-36,   4.03562904e-27,   3.29136288e-33],
       [  8.45176816e-01,   1.68802973e-04,   1.59523712e-04, ...,
          9.51337142e-08,   2.17896790e-09,   5.41938924e-11],
       ..., 
       [  7.17702076e-09,   2.83326158e-11,   8.49875465e-12, ...,
          1.22373298e-04,   3.43891088e-06,   1.51997911e-05],
       [  3.75389546e-11,   1.64705546e-15,   9.30344690e-11, ...,
          1.71727673e-25,   1.00000000e+00,   1.11527534e-28],
       [  2.12678492e-06,   1.22138736e-05,   2.10221046e-07, ...,
          9.61596427e-11,   1.65536402e-08,   5.60533477e-12]], dtype=float32)